# Home Assistant Swap Space Analysis v01

read data collected by linux script swap_cap.sh

https://github.com/deepcoder/useful_code/blob/main/swap_cap.sh


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
#import seaborn as sns
import time
import datetime
import pytz
import matplotlib.ticker as ticker
import matplotlib.dates as dates

import os

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
# better display on retina screens

%config InlineBackend.figure_format = 'retina'

In [4]:
time.tzname

('PST', 'PDT')

In [5]:
# local time zone, because if we are running in docker, timezone is probably UTC, NOTE: does this handle DST???
# http://pytz.sourceforge.net/#localized-times-and-date-arithmetic

local_timezone = pytz.timezone('US/Pacific')

In [6]:
def dateparse (time_in_secs):    
    return datetime.datetime.fromtimestamp(float(time_in_secs))

In [7]:
#this adds commas and limits decimals to 1 places
pd.options.display.float_format = '{:,.1f}'.format

In [8]:
# cp swap_cap*.csv ~/Dropbox/data/ha_swap_space
swap_cap_file = '/home/jovyan/data/ha_swap_space/swap_cap_gen.csv'
# swap_cap_file = '/home/jovyan/data/ha_swap_space/swap_cap_ha.csv'

In [9]:
print('swap_cap_file : %s' % swap_cap_file)
print('swap_cap_file last modified: %s' % time.ctime(os.path.getmtime(swap_cap_file)))
print('swap_cap_file created: %s' % time.ctime(os.path.getctime(swap_cap_file)))

swap_cap_file : /home/jovyan/data/ha_swap_space/swap_cap_gen.csv
swap_cap_file last modified: Wed Jun 30 10:30:12 2021
swap_cap_file created: Wed Jun 30 10:30:12 2021


In [10]:
# read in csv file, convert unit timestamp to pandas data time, this should show in local time zone
headers = ["pid", "swap", "mult", "timestamp", "name"]
datatypes = {'pid:':np.int32, 'swap':np.float64, 'mult':str, 'timestamp':str, 'name':str}
date_cols = ['timestamp']
%time swap_data    = pd.read_csv(swap_cap_file, dtype=datatypes, header=None, names=headers, parse_dates=date_cols, date_parser=dateparse)

CPU times: user 34.5 ms, sys: 4.3 ms, total: 38.8 ms
Wall time: 47.8 ms


In [11]:
swap_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4839 entries, 0 to 4838
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   pid        4839 non-null   int64         
 1   swap       4839 non-null   float64       
 2   mult       4839 non-null   object        
 3   timestamp  4839 non-null   datetime64[ns]
 4   name       4839 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 189.1+ KB


In [12]:
# current swap utilization and maximum size
# swapon --show=USED,SIZE --bytes
#      USED       SIZE
# 441974784 2147479552
current_swap_size = !swapon --show=USED,SIZE --bytes

In [13]:
print(current_swap_size[0] + "\n" + current_swap_size[1])

     USED       SIZE
441974784 2147479552


In [14]:
swap_data.sort_values(by=['pid'],inplace=True)

In [15]:
# linux up time
swap_data[swap_data['pid'] == -1].sort_values(by=['timestamp'], ascending=False).head(10)

,pid,swap,mult,timestamp,name
4648,-1,"140,011.9","""""",2021-06-30 10:30:01,"""swap_cap.sh"""
4458,-1,"136,412.3","""""",2021-06-30 09:30:01,"""swap_cap.sh"""
4268,-1,"132,812.6","""""",2021-06-30 08:30:02,"""swap_cap.sh"""
4083,-1,"129,212.0","""""",2021-06-30 07:30:01,"""swap_cap.sh"""
3896,-1,"125,611.7","""""",2021-06-30 06:30:01,"""swap_cap.sh"""
3709,-1,"122,011.8","""""",2021-06-30 05:30:01,"""swap_cap.sh"""
3524,-1,"118,411.7","""""",2021-06-30 04:30:01,"""swap_cap.sh"""
3362,-1,"114,812.1","""""",2021-06-30 03:30:01,"""swap_cap.sh"""
3203,-1,"111,212.4","""""",2021-06-30 02:30:01,"""swap_cap.sh"""
3043,-1,"107,612.7","""""",2021-06-30 01:30:02,"""swap_cap.sh"""


In [16]:
# processes with largest swap usage
swap_data[swap_data['pid'] != -1].sort_values(by=['swap', 'name', 'timestamp'], ascending=False).head(10)

,pid,swap,mult,timestamp,name
4690,1477,"114,684.0","""kB""",2021-06-30 10:30:01,"""/usr/bin/gnome-shell """
4494,1477,"112,476.0","""kB""",2021-06-30 09:30:01,"""/usr/bin/gnome-shell """
4305,1477,"111,188.0","""kB""",2021-06-30 08:30:02,"""/usr/bin/gnome-shell """
4120,1477,"110,036.0","""kB""",2021-06-30 07:30:01,"""/usr/bin/gnome-shell """
3933,1477,"108,308.0","""kB""",2021-06-30 06:30:01,"""/usr/bin/gnome-shell """
3746,1477,"106,356.0","""kB""",2021-06-30 05:30:01,"""/usr/bin/gnome-shell """
3561,1477,"98,132.0","""kB""",2021-06-30 04:30:01,"""/usr/bin/gnome-shell """
2135,1477,"73,708.0","""kB""",2021-06-29 19:30:01,"""/usr/bin/gnome-shell """
3398,1477,"72,920.0","""kB""",2021-06-30 03:30:01,"""/usr/bin/gnome-shell """
3238,1477,"72,920.0","""kB""",2021-06-30 02:30:01,"""/usr/bin/gnome-shell """


In [17]:
# largest postgresql process swap usage
swap_data[swap_data['name'].str.contains('postgres: user')].sort_values(by=['pid', 'timestamp'], ascending=False).head(10)

,pid,swap,mult,timestamp,name
4668,1356751,948.0,"""kB""",2021-06-30 10:30:01,"""postgres: user homeassistant 192.168.2.242(3..."
4667,1356750,928.0,"""kB""",2021-06-30 10:30:01,"""postgres: user homeassistant 192.168.2.242(3..."
4666,1356749,948.0,"""kB""",2021-06-30 10:30:01,"""postgres: user homeassistant 192.168.2.242(3..."
4665,1356733,948.0,"""kB""",2021-06-30 10:30:01,"""postgres: user homeassistant 192.168.2.242(3..."
4664,1356622,948.0,"""kB""",2021-06-30 10:30:01,"""postgres: user homeassistant 192.168.2.242(6..."
4625,739319,"10,220.0","""kB""",2021-06-30 09:30:01,"""postgres: user homeassistant 192.168.2.242(5..."
4435,739319,"10,216.0","""kB""",2021-06-30 08:30:02,"""postgres: user homeassistant 192.168.2.242(5..."
4245,739319,"10,168.0","""kB""",2021-06-30 07:30:01,"""postgres: user homeassistant 192.168.2.242(5..."
4060,739319,"10,072.0","""kB""",2021-06-30 06:30:01,"""postgres: user homeassistant 192.168.2.242(5..."
3873,739319,"9,072.0","""kB""",2021-06-30 05:30:01,"""postgres: user homeassistant 192.168.2.242(5..."


In [18]:
# largest home assistant python process swap usage
swap_data[swap_data['name'].str.contains('python3 -m')].sort_values(by=['timestamp'], ascending=False).head(10)

,pid,swap,mult,timestamp,name
4811,555981,"24,156.0","""kB""",2021-06-30 10:30:01,"""python3 -m homeassistant --config /config """
4615,555981,"23,864.0","""kB""",2021-06-30 09:30:01,"""python3 -m homeassistant --config /config """
4425,555981,"23,752.0","""kB""",2021-06-30 08:30:02,"""python3 -m homeassistant --config /config """
4235,555981,"23,752.0","""kB""",2021-06-30 07:30:01,"""python3 -m homeassistant --config /config """
4048,555981,"23,620.0","""kB""",2021-06-30 06:30:01,"""python3 -m homeassistant --config /config """
3861,555981,"23,220.0","""kB""",2021-06-30 05:30:01,"""python3 -m homeassistant --config /config """
3675,555981,"21,596.0","""kB""",2021-06-30 04:30:01,"""python3 -m homeassistant --config /config """
3491,555981,"4,508.0","""kB""",2021-06-30 03:30:01,"""python3 -m homeassistant --config /config """
3329,555981,"4,228.0","""kB""",2021-06-30 02:30:01,"""python3 -m homeassistant --config /config """
3170,555981,"4,248.0","""kB""",2021-06-30 01:30:02,"""python3 -m homeassistant --config /config """


In [19]:
# processes with largest swap usage by time
swap_data[swap_data['pid'] != -1].sort_values(by=['timestamp', 'swap'], ascending=False).head(10)

,pid,swap,mult,timestamp,name
4690,1477,"114,684.0","""kB""",2021-06-30 10:30:01,"""/usr/bin/gnome-shell """
4811,555981,"24,156.0","""kB""",2021-06-30 10:30:01,"""python3 -m homeassistant --config /config """
4813,5658,"19,672.0","""kB""",2021-06-30 10:30:01,"""npm """
4790,4815,"18,736.0","""kB""",2021-06-30 10:30:01,"""hb-service """
4707,1664,"13,856.0","""kB""",2021-06-30 10:30:01,"""/usr/bin/dockerd -H fd:// --containerd=/run/..."
4791,4847,"12,952.0","""kB""",2021-06-30 10:30:01,"""homebridge """
4737,1852,"12,780.0","""kB""",2021-06-30 10:30:01,"""/usr/libexec/evolution-data-server/evolution..."
4693,1527,"10,928.0","""kB""",2021-06-30 10:30:01,"""/usr/libexec/ibus-extension-gtk3 """
4814,5746,"10,736.0","""kB""",2021-06-30 10:30:01,"""node-red """
4742,1955,"10,056.0","""kB""",2021-06-30 10:30:01,"""/usr/lib/vino/vino-server """


In [20]:
# get swap use change over time
test_delta  = swap_data[swap_data['pid'] != -1].sort_values(by=['pid','timestamp'], ascending=True)
test_delta['diff'] = test_delta.groupby(['pid'])['swap'].diff()

In [21]:
# processes with largest change in swap usage over time
test_delta[test_delta['diff'].notnull()].sort_values(by=['diff'], ascending=False).head(10)

,pid,swap,mult,timestamp,name,diff
3561,1477,"98,132.0","""kB""",2021-06-30 04:30:01,"""/usr/bin/gnome-shell ""","25,212.0"
3675,555981,"21,596.0","""kB""",2021-06-30 04:30:01,"""python3 -m homeassistant --config /config ""","17,088.0"
426,1477,"51,064.0","""kB""",2021-06-29 07:30:01,"""/usr/bin/gnome-shell ""","16,004.0"
3682,5876,"16,200.0","""kB""",2021-06-30 04:30:01,"""/opt/conda/bin/python3.8 /opt/conda/bin/jupy...","14,480.0"
136,6422,"10,308.0","""kB""",2021-06-29 04:30:01,"""python3 -m homeassistant --config /config ""","10,084.0"
130,5658,"10,284.0","""kB""",2021-06-29 04:30:01,"""npm ""","10,044.0"
3680,5746,"9,912.0","""kB""",2021-06-30 04:30:01,"""node-red ""","8,728.0"
3678,566013,"10,016.0","""kB""",2021-06-30 04:30:01,"""postgres: user homeassistant 192.168.2.242(5...","8,640.0"
1414,1477,"66,776.0","""kB""",2021-06-29 14:30:01,"""/usr/bin/gnome-shell ""","8,404.0"
3746,1477,"106,356.0","""kB""",2021-06-30 05:30:01,"""/usr/bin/gnome-shell ""","8,224.0"


In [22]:
# home assistant python swap size change over time
test_delta[test_delta['name'].str.contains('python3 -m')].sort_values(by=['timestamp'], ascending=False)

,pid,swap,mult,timestamp,name,diff
4811,555981,"24,156.0","""kB""",2021-06-30 10:30:01,"""python3 -m homeassistant --config /config """,292.0
4615,555981,"23,864.0","""kB""",2021-06-30 09:30:01,"""python3 -m homeassistant --config /config """,112.0
4425,555981,"23,752.0","""kB""",2021-06-30 08:30:02,"""python3 -m homeassistant --config /config """,0.0
4235,555981,"23,752.0","""kB""",2021-06-30 07:30:01,"""python3 -m homeassistant --config /config """,132.0
4048,555981,"23,620.0","""kB""",2021-06-30 06:30:01,"""python3 -m homeassistant --config /config """,400.0
3861,555981,"23,220.0","""kB""",2021-06-30 05:30:01,"""python3 -m homeassistant --config /config ""","1,624.0"
3675,555981,"21,596.0","""kB""",2021-06-30 04:30:01,"""python3 -m homeassistant --config /config ""","17,088.0"
3491,555981,"4,508.0","""kB""",2021-06-30 03:30:01,"""python3 -m homeassistant --config /config """,280.0
3329,555981,"4,228.0","""kB""",2021-06-30 02:30:01,"""python3 -m homeassistant --config /config """,-20.0
3170,555981,"4,248.0","""kB""",2021-06-30 01:30:02,"""python3 -m homeassistant --config /config """,0.0


In [23]:
# postgresql home assistant query processes swap size change over time
test_delta[test_delta['name'].str.contains('postgres: user')].sort_values(by=['pid','timestamp'], ascending=False).head(40)

,pid,swap,mult,timestamp,name,diff
4668,1356751,948.0,"""kB""",2021-06-30 10:30:01,"""postgres: user homeassistant 192.168.2.242(3...",nan
4667,1356750,928.0,"""kB""",2021-06-30 10:30:01,"""postgres: user homeassistant 192.168.2.242(3...",nan
4666,1356749,948.0,"""kB""",2021-06-30 10:30:01,"""postgres: user homeassistant 192.168.2.242(3...",nan
4665,1356733,948.0,"""kB""",2021-06-30 10:30:01,"""postgres: user homeassistant 192.168.2.242(3...",nan
4664,1356622,948.0,"""kB""",2021-06-30 10:30:01,"""postgres: user homeassistant 192.168.2.242(6...",nan
4625,739319,"10,220.0","""kB""",2021-06-30 09:30:01,"""postgres: user homeassistant 192.168.2.242(5...",4.0
4435,739319,"10,216.0","""kB""",2021-06-30 08:30:02,"""postgres: user homeassistant 192.168.2.242(5...",48.0
4245,739319,"10,168.0","""kB""",2021-06-30 07:30:01,"""postgres: user homeassistant 192.168.2.242(5...",96.0
4060,739319,"10,072.0","""kB""",2021-06-30 06:30:01,"""postgres: user homeassistant 192.168.2.242(5...","1,000.0"
3873,739319,"9,072.0","""kB""",2021-06-30 05:30:01,"""postgres: user homeassistant 192.168.2.242(5...","3,296.0"
